# Package/Unpackage Network

This notebook includes a function designed to save networks as CSV files and convert them back to NetCDF format.

To automate the conversion of CSV files into a PyPSA network and save them in the appropriate folder for the Snakemake workflow, follow these steps:

1. Download the zip file for the scenarios.
2. Unzip it and place the contents in the `raw_scenario_csv_folders` folder.

Go to the next cell

In [ ]:
import pypsa
import os.path
import yaml
import os

import warnings

warnings.filterwarnings("ignore")

config = {}
for c in ["kpi","form","default"]:
    with open(f"../config/config.{c}.yaml") as file:
        config_append = yaml.safe_load(file)

    config.update(config_append)

with open("../config/scenarios.form.yaml") as file:
    config_scenarios = yaml.safe_load(file)

def import_scenarios_from_netcdf(scenarios):
# Creating the dictionary
    scenarios_network = {}
    
    for scenario in scenarios:
        if 'delay' in scenario:
            postnetwork = "base_s_52_lv1.012___2035.nc"
        else:
            postnetwork = "base_s_52_lv1.11___2035.nc"

        try:
            n = pypsa.Network(f"../results/{scenario}/{scenario}/postnetworks/{postnetwork}")
            scenarios_network.update({scenario:n})
        except:
            print("The " + scenario + " import from netCDF was unsuccessful.")
        
    return scenarios_network

def export_scenarios_to_netcdf(n_scenarios):
    for scenario, n in n_scenarios.items():
        if 'delay' in scenario:
            postnetwork = "base_s_52_lv1.012___2035.nc"
        else:
            postnetwork = "base_s_52_lv1.11___2035.nc"
        
        path = f"../results/{scenario}/{scenario}/postnetworks/"
        if not os.path.exists(path):
            print(path + " does not exist, creating it now.")
            os.makedirs(path)
        n.export_to_netcdf(path + postnetwork)
        print("The " + scenario + " has been exported to NetCDF in " + path)

def export_scenarios_to_csv(n_scenarios, custom_path=""):
    for scenario, n in n_scenarios.items():
        if not os.path.exists(custom_path):
            print(custom_path + " does not exist, creating it now.")
            os.makedirs(custom_path)
        n.export_to_csv_folder(custom_path + scenario)
        print("The " + scenario + " has been exported to the CSV.")

def import_scenarios_from_csv(scenarios,custom_path=""):
# Creating the dictionary
    scenarios_network = {}
    
    for scenario in scenarios:
        network_path = custom_path + str(scenario)
        try:
            n = pypsa.Network()
            n.import_from_csv_folder(network_path)
            scenarios_network.update({scenario:n})
            print("The " + scenario + " has been imported from the CSV.")
        except:
            print("The " + scenario + " import from the CSV was unsuccessful.")
        
    return scenarios_network

print("here are the list of all scenarios according to scenarios.form.yaml:")
print(list(config_scenarios.keys()))

## Converting csvs to PyPSA Network to NetCDF 

3. Select the scenarios you want to import and it will create the pypsa network
4. (Next cell) save the pypsa network in the appropriate folder for the snakemake workflow



In [ ]:
# select all scenarios that you want to include
scenarios_chosen = ['baseline-mds', 'baseline-nomds', 'mid-capex-mds', 'low-capex-mds',
                    'cy2010-mds', 'cy2010-nomds', 'cy2012-mds', 'cy2012-nomds', 'cy1996-mds', 'cy1996-nomds', 
                    'nogasppde-mid-capex-mds', 'nogasppde-low-capex-mds', 'nogasppde-nomds', 'delayed-nep-mid-capex-mds', 
                    'delayed-nep-low-capex-mds', 'delayed-nep-nomds', 'gasprice-peak-mid-capex-mds', 'gasprice-peak-low-capex-mds', 
                    'gasprice-peak-nomds', 'cy2010-mid-capex-mds', 'cy2010-low-capex-mds', 'cy2012-mid-capex-mds', 'cy2012-low-capex-mds']

directory = "raw_scenario_csv_folders/"
scenarios_list = set(scenarios_chosen).intersection(os.listdir(directory))

print(f"importing {list(scenarios_list)}")
n_scenarios = import_scenarios_from_csv(scenarios_list,custom_path=directory)

In [ ]:
export_scenarios_to_netcdf(n_scenarios)

# Archive

**Import the network from NetCDF**

In [ ]:
n_scenarios = import_scenarios_from_netcdf(scenarios_list)

**Export the network to CSV**

In [ ]:
directory = "raw_scenario_csv_folders/"
export_scenarios_to_csv(n_scenarios, custom_path = directory)

**Import the network from CSV**

In [ ]:
n_scenarios = import_scenarios_from_csv(scenarios_list)

**Export the network to NetCDF**

In [ ]:
export_scenarios_to_netcdf(n_scenarios)